In [1]:
import pandas as pd

In [2]:
filtered_clinical = pd.read_csv('june22_clinical with ndvi norm.csv')
filtered_daily = pd.read_csv('june22_daily ema with ndvi norm.csv')
clinical_daily_final_merge = pd.read_csv('june22_merge clinical and daily.csv')

# filtered_clinical = pd.read_csv('june22_clinical with ndvi norm_year.csv')
# filtered_daily = pd.read_csv('june22_daily ema with ndvi norm_year.csv')
# clinical_daily_final_merge = pd.read_csv('june22_merge clinical and daily_year.csv')


In [3]:
# -------- Step 1: Divide it into the individual levels (ndvi >=0.5) ---------

In [ ]:
######### US CLINICAL #########
us_clinical = filtered_clinical.copy()
us_chr = us_clinical[us_clinical['Phenotype'] == 'CHR'].copy()
us_hc = us_clinical[us_clinical['Phenotype'] == 'HC'].copy()

us_high_all = filtered_clinical[filtered_clinical['ndvi_v4_mean'] >= 0.5].copy()
us_low_all = filtered_clinical[filtered_clinical['ndvi_v4_mean'] < 0.5].copy()

us_high_chr = us_high_all[us_high_all['Phenotype'] == 'CHR'].copy()
us_low_chr = us_low_all[us_low_all['Phenotype'] == 'CHR'].copy()

us_high_hc = us_high_all[us_high_all['Phenotype'] == 'HC'].copy()
us_low_hc = us_low_all[us_low_all['Phenotype'] == 'HC'].copy()


#### EMA + Clinical ####
us_merged_daily_total =clinical_daily_final_merge.copy()
us_merged_daily_chr = clinical_daily_final_merge[clinical_daily_final_merge['Phenotype'] == 'CHR'].copy()
us_merged_daily_hc = clinical_daily_final_merge[clinical_daily_final_merge['Phenotype'] == 'HC'].copy()

us_merged_daily_total_high = us_merged_daily_total[us_merged_daily_total['ndvi_v4'] >= 0.5].copy()
us_merged_daily_chr_high = us_merged_daily_total_high[us_merged_daily_total_high['Phenotype'] == 'CHR'].copy()
us_merged_daily_hc_high = us_merged_daily_total_high[us_merged_daily_total_high['Phenotype'] == 'HC'].copy()


us_merged_daily_total_low = us_merged_daily_total[us_merged_daily_total['ndvi_v4'] < 0.5].copy()
us_merged_daily_chr_low = us_merged_daily_total_low[us_merged_daily_total_low['Phenotype'] == 'CHR'].copy()
us_merged_daily_hc_low = us_merged_daily_total_low[us_merged_daily_total_low['Phenotype'] == 'HC'].copy()


In [ ]:
## US Maps Index

#### Clinical ####
# us_clinical
# us_chr
# us_hc

# us_high_all
# us_low_all

# us_high_chr
# us_low_chr

# us_high_hc
# us_low_hc

#### EMA + Clinical ####
# us_merged_daily_total
# us_merged_daily_chr
# us_merged_daily_hc

# us_merged_daily_total_high
# us_merged_daily_chr_high
# us_merged_daily_hc_high

# us_merged_daily_total_low
# us_merged_daily_chr_low
# us_merged_daily_hc_low

In [ ]:
# ----------- Step 2: Mediating Variables -------------

### Varaibles:
# 1. EMA passive phenotyping: Sleep duration, Home time, Screen time, Entropy
# 2. Sociodemographic Information: Income, Education, Housing, Sex, Age


In [ ]:
# Re-import needed packages after code execution state reset
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

setattr(pd, "Float64Index", pd.Index) # Hacky fix for ImportError.
setattr(pd, "Int64Index", pd.Index)
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib.patches import FancyArrowPatch

from math import sqrt
from scipy.stats import norm


def run_mediation_moderation(
    data,
    mediate_var='percent_home_norm',
    dv_var='control_norm',
    iv_var='ndvi_v4',
):
    
    
    # ==== Step 1: Prepare data ====
    df = data.copy()
    # Z-score transformation
    df["ndvi_z"] = (df[iv_var] - df[iv_var].mean()) / df[iv_var].std()
    df["mediating_z"] = (df[mediate_var] - df[mediate_var].mean()) / df[mediate_var].std()

    
    # ==== Step 2: Fit Mediation Models ====
    # a path: IV → Mediator
    a_model = smf.ols(f"{mediate_var} ~ {iv_var}", data=df).fit()
    print("A summary")
    print(a_model.summary())
    print("-" * 30)
    a = a_model.params[iv_var]
    a_p = a_model.pvalues[iv_var]

    # b and c′ paths: Mediator + IV → DV
    b_model = smf.ols(f"{dv_var} ~ {iv_var} + {mediate_var}", data=df).fit()
    print("B summary")
    print(b_model.summary())
    print("-" * 30)
    b = b_model.params[mediate_var]
    b_p = b_model.pvalues[mediate_var]
    c_prime = b_model.params[iv_var]
    c_prime_p = b_model.pvalues[iv_var]

    # Total effect c path: IV → DV
    c_model = smf.ols(f"{dv_var} ~ {iv_var}", data=df).fit()
    print("C summary")
    print(c_model.summary())
    print("-" * 30)
    c = c_model.params[iv_var]
    c_p = c_model.pvalues[iv_var]

    # Indirect effect
    indirect = a * b
    
     # Get standard errors from models
    se_a = a_model.bse[iv_var]
    se_b = b_model.bse[mediate_var]
    # Sobel standard error
    se_indirect = sqrt(b**2 * se_a**2 + a**2 * se_b**2)
    # Sobel z-value
    z_sobel = indirect / se_indirect
    # Two-tailed p-value
    indirect_p = 2 * (1 - norm.cdf(abs(z_sobel)))
    # Print or use this in your summary
    print(f"Indirect Effect p-value (Sobel test): {indirect_p:.4f}")


    # ==== Step 3: Mediation Plot ====

    def p_to_stars(p):
        if p < 0.001: return "***"
        elif p < 0.01: return "**"
        elif p < 0.05: return "*"
        else: return ""

    plt.figure(figsize=(8, 5))
    ax = plt.gca()

    # Nodes
    ax.text(0.1, 0.5, iv_var, bbox=dict(facecolor="lightblue", edgecolor="black"), fontsize=12)
    ax.text(0.5, 0.8, mediate_var, bbox=dict(facecolor="lightyellow", edgecolor="black"), fontsize=12)
    ax.text(0.9, 0.5, dv_var, bbox=dict(facecolor="lightgreen", edgecolor="black"), fontsize=12)

    # Arrows
    def draw_arrow(xy_start, xy_end, text, fontsize=11):
        ax.annotate("", xy=xy_end, xytext=xy_start,
                    arrowprops=dict(arrowstyle="->", lw=2))
        mid_x = (xy_start[0] + xy_end[0]) / 2
        mid_y = (xy_start[1] + xy_end[1]) / 2
        ax.text(mid_x, mid_y, text, fontsize=fontsize)

    # Paths: c′, a, b
    draw_arrow((0.18, 0.5), (0.82, 0.5), f"{c_prime:.2f} {p_to_stars(c_prime_p)}")  # direct
    draw_arrow((0.18, 0.5), (0.5, 0.78), f"{a:.2f} {p_to_stars(a_p)}")              # a path
    draw_arrow((0.5, 0.78), (0.82, 0.5), f"{b:.2f} {p_to_stars(b_p)}")              # b path

    plt.title("Mediation Diagram", fontsize=14)
    ax.axis("off")
    plt.tight_layout()
    plt.show()
    
    print("=== Mediation Summary ===")
    print(f"a ({iv_var} → {mediate_var}): {a:.3f} {p_to_stars(a_p)}")
    print(f"b ({mediate_var} → {dv_var}): {b:.3f} {p_to_stars(b_p)}")
    print(f"Indirect Effect (a×b): {indirect:.3f} {p_to_stars(indirect_p)}")
    print(f"Direct Effect (c′: {iv_var} → {dv_var} controlling for {mediate_var}): {c_prime:.3f} {p_to_stars(c_prime_p)}")
    print(f"Total Effect (c: {iv_var} → {dv_var}): {c:.3f} {p_to_stars(c_p)}")

   
    
    # ==== Step 4: Moderation Plot ====

    # Create interaction term
    df["interaction"] = df[iv_var] * df[mediate_var]
    mod_model = smf.ols(f"{dv_var} ~ {iv_var} * {mediate_var}", data=df).fit()
    interaction_p = mod_model.pvalues[f"{iv_var}:{mediate_var}"]
    

    print(mod_model.summary())
    print(f"Moderation Interaction Term p-value: {interaction_p:.3f} {p_to_stars(interaction_p)}")
    
    # ==== Step 5: Summary Table ====
    summary_df = pd.DataFrame({
        'Path': [
            f'a path',               
            f'b path',              
            f'c′ path (direct)',           
            f'c path (total)',            
            f'Indirect Effect (a×b)',              
            f'moderation'  
        ],
        'Coefficient': [
            round(a, 3),
            round(b, 3),
            round(c_prime, 3),
            round(c, 3),
            round(indirect, 3),
            round(mod_model.params.get(f"{iv_var}:{mediate_var}", np.nan), 3)
        ],
        'p-value': [
            round(a_p, 3),
            round(b_p, 3),
            round(c_prime_p, 3),
            round(c_p, 3),
            round(indirect_p, 3),
            round(interaction_p, 3)
        ],
        'Significance': [
            p_to_stars(a_p),
            p_to_stars(b_p),
            p_to_stars(c_prime_p),
            p_to_stars(c_p),
            p_to_stars(indirect_p),
            p_to_stars(interaction_p)
        ]
    })

    print("\n=== Summary Table ===")
    return summary_df



In [ ]:
# -------- Sample Data Analysis  ---------

In [ ]:
# run_mediation_moderation(
#     data = us_merged_daily_chr,
#     mediate_var='percent_home_norm',
#     dv_var='bprs_total_m2_norm',
#     iv_var='ndvi_v4',
# )


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from math import sqrt
from scipy.stats import norm

def multiple_regression(
    data,
    mediate_var='percent_home_norm',  # categorical
    dv_var='control_norm',
    iv_var='ndvi_v4',
):
    df = data.copy()

    b_model = smf.ols(f"{dv_var} ~ {iv_var} + C({mediate_var})", data=df).fit()
    print(b_model.summary())
    print("-" * 30)

    

In [ ]:
##################### SEM ########################

In [ ]:
def run_mediation_sem(df, iv_var, mediate_var, dv_var):
   
    def p_to_stars(p):
        if p < 0.001: return "***"
        elif p < 0.01: return "**"
        elif p < 0.05: return "*"
        else: return ""

    # Copy relevant data
    df_model = df[[iv_var, mediate_var, dv_var]].copy()

    df_model = df[[iv_var, mediate_var, dv_var]].dropna().copy()


    # Z-score all variables
    df_model['iv_z'] = zscore(df_model[iv_var])
    df_model['med_z'] = zscore(df_model[mediate_var].astype(float))
    df_model['dv_z'] = zscore(df_model[dv_var])

    # Define SEM model syntax
    model_desc = """
    med_z ~ iv_z
    dv_z ~ med_z + iv_z
    """

    # Fit model
    model = Model(model_desc)
    model.fit(df_model)

    # Extract parameter estimates
    estimates_df = model.inspect()

    # Extract values for paths
    a_row = estimates_df[(estimates_df['lval'] == 'med_z') & (estimates_df['rval'] == 'iv_z')]
    b_row = estimates_df[(estimates_df['lval'] == 'dv_z') & (estimates_df['rval'] == 'med_z')]
    c_row = estimates_df[(estimates_df['lval'] == 'dv_z') & (estimates_df['rval'] == 'iv_z')]

    # Coefficients
    a = a_row['Estimate'].values[0]
    b = b_row['Estimate'].values[0]
    c_prime = c_row['Estimate'].values[0]

    # p-values
    a_p = a_row['p-value'].values[0]
    b_p = b_row['p-value'].values[0]
    c_p = c_row['p-value'].values[0]

    # Effects
    indirect = a * b
    total = indirect + c_prime

    # Summary table
    summary_df = pd.DataFrame({
        'Effect': [
            'a (IV → Mediator)',
            'b (Mediator → DV)',
            "c′ (Direct IV → DV)",
            'Indirect (a*b)',
            'Total (a*b + c′)'
        ],
        'Estimate': [
            round(a, 3),
            round(b, 3),
            round(c_prime, 3),
            round(indirect, 3),
            round(total, 3)
        ],
        'p-value': [
            round(a_p, 4),
            round(b_p, 4),
            round(c_p, 4),
            '',  # p for indirect not available without bootstrapping
            ''
        ],
        'Significance': [
            p_to_stars(a_p),
            p_to_stars(b_p),
            p_to_stars(c_p),
            '',
            ''
        ]
    })

    return summary_df


In [ ]:
# -------- Data Analysis Samples ---------

In [ ]:
# run_mediation_sem(
#     df=us_merged_daily_chr,
#     iv_var='ndvi_v4',
#     mediate_var='education',
#     dv_var='bprs_total_m2_norm'
# )


In [ ]:
# --------- Data Analysis Visualizations ----------

In [ ]:
# --------- Significan Mean Difference (CHR + HC) -----------

In [ ]:
def plot_group_comparison_multiple(df_list, scales, labels_list):
    """
    Plot grouped bar graph with error bars and significance markers comparing CHR vs HC groups across multiple timepoints.
    
    Parameters:
    - df_list: list of pandas DataFrames for different timepoints (bl, m1, m2)
    - scales: dict of scales {label: column_name}
    - labels_list: list of str labels for each timepoint (["Baseline", "Month 1", "Month 2"])
    
    Returns:
    - results_dict: {label: t_test_df} for each timepoint
    - sem_dict: {label: sem_df} for each timepoint
    """
    num_timepoints = len(df_list)
    labels = [label.upper() for label in scales.keys()]
    x = np.arange(len(labels))

    fig, ax = plt.subplots(figsize=(22, 10))

    bar_width = 0.2
    offset = -bar_width * (num_timepoints - 1) / 2

    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # blue, orange, green

    results_dict = {}
    sem_dict = {}

    for idx, (df, timepoint_label) in enumerate(zip(df_list, labels_list)):
        df = df.copy()
        df['Phenotype_group'] = df['Phenotype'].apply(lambda x: 'CHR' if x == 'CHR' else 'HC')

        t_test_results = []
        chr_errors = []
        hc_errors = []

        for scale_label, col_name in scales.items():
            chr_group = df[df['Phenotype_group'] == 'CHR'][col_name].dropna()
            hc_group = df[df['Phenotype_group'] == 'HC'][col_name].dropna()

            t_stat, p_val = ttest_ind(chr_group, hc_group, equal_var=False)

            t_test_results.append({
                'n_chr': len(chr_group),
                'n_hc': len(hc_group),
                'Scale': scale_label.upper(),
                'CHR Mean': round(chr_group.mean(), 3),
                'HC Mean': round(hc_group.mean(), 3),
                't-statistic': round(t_stat, 3),
                'p-value': round(p_val, 4),
                'Significance': '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else ''
            })

            chr_errors.append(round(sem(chr_group), 3))
            hc_errors.append(round(sem(hc_group), 3))

        t_test_df = pd.DataFrame(t_test_results)
        sem_df = pd.DataFrame({
            'Scale': list(scales.keys()),
            f'{timepoint_label} CHR SEM': chr_errors,
            f'{timepoint_label} HC SEM': hc_errors
        })

        results_dict[timepoint_label] = t_test_df
        sem_dict[timepoint_label] = sem_df

        chr_means = t_test_df['CHR Mean']
        hc_means = t_test_df['HC Mean']

        ax.bar(x + offset + idx * bar_width, chr_means, bar_width, 
               yerr=chr_errors, capsize=4, label=f'CHR - {timepoint_label}', alpha=0.8, color=colors[idx])
        ax.bar(x + offset + idx * bar_width, hc_means, bar_width, 
               yerr=hc_errors, capsize=4, label=f'HC - {timepoint_label}', alpha=0.3, color=colors[idx])

    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_ylim(0, 1)
    ax.set_ylabel('Normalized Mean Score')
    ax.set_title('Symptom Scores by Group (CHR vs HC) Across Timepoints')
    ax.legend()
    plt.tight_layout()
    plt.show()

    return results_dict, sem_dict


In [ ]:
# ------ Simpsons Paradox Plot ---------

In [ ]:
def simpsons_paradox_plot(df, x_col, y_col, group_col='Greenspace'):
    # Drop NA
    data = df[[x_col, y_col, group_col]].dropna()

    # Use lmplot to plot group-specific regression
    g = sns.lmplot(
        data=data,
        x=x_col,
        y=y_col,
        hue=group_col,
        palette={'High': 'green', 'Low': 'gold'},
        markers=['o', 's'],
        height=10,
        aspect=1.4,
        legend=False
    )

    ax = g.ax

    # Plot overall regression line WITHOUT label first
    overall_model = sm.OLS.from_formula(f"{y_col} ~ {x_col}", data=data).fit()
    x_vals = np.linspace(data[x_col].min(), data[x_col].max(), 100)
    y_vals = overall_model.params['Intercept'] + overall_model.params[x_col] * x_vals
    overall_line, = ax.plot(x_vals, y_vals, color='black', linestyle='--')

    # Get existing legend handles/labels
    handles, labels = ax.get_legend_handles_labels()

    # Append just one 'Overall Trend' entry
    handles.append(overall_line)
    labels.append('Overall Trend')

    ax.legend(handles, labels)

    ax.set_title(f"{y_col} vs {x_col} by Greenspace Group")
    plt.tight_layout()
    plt.show()

    # Collect regression slope results into a list
    regression_summary = []

    for group in data[group_col].unique():
        sub_data = data[data[group_col] == group]
        model = sm.OLS.from_formula(f"{y_col} ~ {x_col}", data=sub_data).fit()
        regression_summary.append({
            'Group': group,
            'Beta': round(model.params[x_col], 4),
            'P-value': round(model.pvalues[x_col], 4)
        })

    # Add overall regression
    regression_summary.append({
        'Group': 'Overall',
        'Beta': round(overall_model.params[x_col], 4),
        'P-value': round(overall_model.pvalues[x_col], 4)
    })

    regression_df = pd.DataFrame(regression_summary)
    return regression_df


In [ ]:
# -------- Sample Data Analysis ---------

In [ ]:
# simpsons_paradox_plot(us_merged_daily_chr, x_col='ndvi_v4', y_col='bprs_total_baseline_norm')
# simpsons_paradox_plot(us_merged_daily_chr, x_col='ndvi_v4', y_col='bprs_total_m2_norm')

